In [5]:
# Step 0. Load libraries and modules
# Dataframes and matrices ----------------------------------------------
import pandas as pd
import numpy as np
# Graphics -------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# Statistics -----------------------------------------------------------
import statsmodels.formula.api as smf
from statsmodels.api import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
# Machine learning -----------------------------------------------------
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
# Preprocessing --------------------------------------------------------
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# Metrics --------------------------------------------------------------
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

In [6]:
# Step 1. Load the dataset
# Source: https://www.kaggle.com/code/sagarvarandekar/medical-insurance-regression-analysis-and-predict/notebook
df_raw = pd.read_csv('../data/raw/insurance.csv')

In [8]:
# 1.1 Get basic info
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [9]:
# 1.2 Take a sample
df_raw.sample(5)

,age,sex,bmi,children,smoker,region,charges
417,36,female,22.600,2,yes,southwest,18608.26200
1296,18,male,26.125,0,no,northeast,1708.92575
1251,19,male,19.800,0,no,southwest,1241.56500
1010,48,female,22.800,0,no,southwest,8269.04400
601,51,male,31.635,0,no,northwest,9174.13565


In [12]:
# 1.3 Transform if needed
df_transformed = df_raw.copy()
df_transformed['sex'] = pd.Categorical(df_transformed['sex'])
df_transformed['smoker'] = pd.Categorical(df_transformed['smoker'])
df_transformed['region'] = pd.Categorical(df_transformed['region'])
df_transformed['charges'] = pd.Categorical(df_transformed['charges'])

df = df_transformed.copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       1338 non-null   int64   
 1   sex       1338 non-null   category
 2   bmi       1338 non-null   float64 
 3   children  1338 non-null   int64   
 4   smoker    1338 non-null   category
 5   region    1338 non-null   category
 6   charges   1338 non-null   category
dtypes: category(4), float64(1), int64(2)
memory usage: 81.2 KB


In [14]:
# Step 2. Make a basic EDA
x = df.drop(df['charges'], axis=1)
y = df.charges
X_train,X_test, y_train, y_test = train_test_split(x,y,random_state=28)

KeyError: '[16884.924, 1725.5523, 4449.462, 21984.47061, 3866.8552, 3756.6216, 8240.5896, 7281.5056, 6406.4107, 28923.13692, 2721.3208, 27808.7251, 1826.843, 11090.7178, 39611.7577, 1837.237, 10797.3362, 2395.17155, 10602.385, 36837.467, 13228.84695, 4149.736, 1137.011, 37701.8768, 6203.90175, 14001.1338, 14451.83515, 12268.63225, 2775.19215, 38711.0, 35585.576, 2198.18985, 4687.797, 13770.0979, 51194.55914, 1625.43375, 15612.19335, 2302.3, 39774.2763, 48173.361, 3046.062, 4949.7587, 6272.4772, 6313.759, 6079.6715, 20630.28351, 3393.35635, 3556.9223, 12629.8967, 38709.176, 2211.13075, 3579.8287, 23568.272, 37742.5757, 8059.6791, 47496.49445, 13607.36875, 34303.1672, 23244.7902, 5989.52365, 8606.2174, 4504.6624, 30166.61817, 4133.64165, 14711.7438, 1743.214, 14235.072, 6389.37785, 5920.1041, 17663.1442, 16577.7795, 6799.458, 11741.726, 11946.6259, 7726.854, 11356.6609, 3947.4131, 1532.4697, 2755.02095, 6571.02435, 4441.21315, 7935.29115, 37165.1638, 11033.6617, 39836.519, 21098.55405, 4

In [ ]:
# 2.1 Split the dataset so to avoid bias
df_train = pd.concat([X_train,y_train],axis=1)

In [ ]:
# 2.2 Join the train sets to ease insights

In [ ]:
# 2.3 Get basic info


In [ ]:
# 2.4 Take a sample
df_train.sample(10)

In [ ]:
# 2.5 Describe the numerical and date variables
df_train.describe()

In [ ]:
# 2.6 Describe categorical variables
df_train.describe(include='category')


In [ ]:
# 2.7 Perform univariate analysis - histograms
X_train.hist(figsize=(8,8), alpha=0.5)
plt.show()

In [ ]:
# 2.8 Perform univariate analysis - boxplots
fig, ax = plt.subplots(1,3,figsize=(8,3))
axs = ax.flatten()
for i in range(len(axs)):
    X_train._get_numeric_data().iloc[:,i].to_frame().boxplot(ax=axs[i])

In [ ]:
# 2.9 Perform bivariate analysis - numericals
pd.plotting.scatter_matrix(X_train,diagonal='kde',figsize=(8,8), c='Violet')
plt.show()
#se ven manchas no hay asociaion, deberia ver un ovalo para ver la campana gausiana


In [ ]:
# 2.10 Perform bivariate analysis per category
sns.pairplot(data=X_train, hue='sex')
plt.show()

#sigo sin ver nada entre separacion entre hombre y mujeres

In [ ]:
sns.pairplot(data=X_train, hue='smoker')
plt.show()

#en smoker si se ve algo de separacion,

In [ ]:
sns.pairplot(data=X_train, hue='region')
plt.show()

#tampoco se ve nada de separacion

In [ ]:
# 2.11 Perform bivariate categorical analysis 
sns.countplot(data=X_train,x='sex',hue='smoker')
plt.show()

In [ ]:
sns.countplot(data=X_train,x='region',hue='smoker')
plt.show()

In [ ]:
sns.countplot(data=X_train,x='region',hue='sex')
plt.show()
#no se ve nada q llame la atencion, que haga la diferencia fuerte

In [ ]:
# 2.10 Perform correlation analysis - Pearson or Point Biserial
X_train.corr().style.background_gradient(cmap='Blues') #lo hacemos en las variables numericas

#si encontramos correlacion hay q sacar la columna problematica o se hace una ??? escuchar de nuevo la grabacion


In [ ]:
# 2.11 Perform multivariate analysis - VIF
# https://thecodingbot.com/vif-implementation-in-python/
# https://github.com/statsmodels/statsmodels/issues/2376
# https://www.statology.org/how-to-calculate-vif-in-python/
# See what happens when no constant is added!

#imaginamos q la var dependiente charge no existe

vif = pd.DataFrame()
X_numeric = X_train._get_numeric_data()
X_numeric = add_constant(X_numeric)
vif['features'] = X_numeric.columns
vif['Vif'] = [variance_inflation_factor(X_numeric.values,i) for i in range(X_numeric.shape[1])]
vif

#nos importa el vif menor de 5, como todas estan en 1 no tenemos multicolinealidad
#si es > 10 tengo una var con alta multicolinealidad, tengo q sacarla transfomarla

In [ ]:
# 2.12 Analyze effect on response variable (DV)
#lo vemos con la correlacion
df_train.corr().sytle.background_gradient(cmap='Blues')

#algo me dice

In [ ]:
sns.pairplot(data=df_train)
plt.show()

In [ ]:
sns.pairplot(data=df_train,hue='sex')
plt.show()

In [ ]:
sns.pairplot(data=df_train,hue='smoker')
plt.show()

In [ ]:
sns.pairplot(data=df_train,hue='region')
plt.show()

#hasta aca tenemos un buen analisis y se puede ver la variable q puede tener peso

In [ ]:
# Step 3. Create and evaluate model - Define your approach and goals
# A. Try out Linear Regression, use Statsmodels to verify assumptions
# B. Try out Tree Regression
# C. Try out Random Forest regression
# D. Compare results based on metrics reduction
# Current evidence: Some linear relation between response, age and bmi
# stratified by smoker condition

#mi evidencia es q he visto una ... escuchar q fue lo q dijo para anotar !


In [ ]:
# 3.1 Create your pipeline processing
# https://stackoverflow.com/questions/61641852/
# https://jaketae.github.io/study/sklearn-pipeline/

cat_cols = X_train.select_dtypes(include='category').columns #trae todas las columnas categoricas, hago one hat encoding
num_cols = X_train.select_dtypes(include='number').columns #trae las numericas

#tenemos q llevar esto a una matriz numeric, Pipeline es una secuencia de pasos q luego voy a aplicarle a algo, el q dice _d deja las categoras en n-1 columnas, q es el q uso para la regresion
cat_transformer_d = Pipeline(steps=[('onehot',OneHotEncoder(drop='first',sparse=False))]) #codifica lo q entra , para datos categoricos, caso de sexo, hombre=0, mujer=1
cat_transformer_nd = Pipeline(steps=[('onehot',OneHotEncoder(sparse=False))]) #para usar en random forest y decision tree, no borro nada, uso todo
num_transformer = Pipeline(steps=['scaller',MinMaxScaler()])
preprocessor_d = ColumnTransformer(transformers=[('num',num_transformer,num_cols),('cat',cat_transformer_d,cat_cols)]) #pre procesador , de trans numerico y categorica con var de referencia
preprocessor_nd = ColumnTransformer(transformers=[('num',num_transformer,num_cols),('cat',cat_transformer_nd,cat_cols)]) 
encode_data_d = Pipeline(steps=[('preprocessor',preprocessor_d)])
encode_data_nd = Pipeline(steps=[('preprocessor',preprocessor_nd)])


In [ ]:
# 3.2 Perform model fit - A
lreg= Pipeline(steps=[('preprocessor',preprocessor_d),('regressor',LinearRegression())])
lreg.fit(X_train,y_train)
print(f'R2 Score: {lreg.score(X_train,y_train)}') #Rcuadrado, da 76.1%, 

In [ ]:
# 3.2.1 Calculate metrics on model A
#vamos a ver como nos fue con el test
y_pred = lreg.predict(X_test)
print(f'R2 score: {r2_score(y_test,y_pred)}')
print(f'Mean squared error MSE: {mean_squared_error(y_test,y_pred)}')
print(f'Root Mean squared error RMSE: {np.sqrt(mean_squared_error(y_test,y_pred))}')

In [ ]:
plt.scatter(x=y_test,y=y_pred)
plt.plot([0,40000],[0,40000],color='Red')
plt.xlabel('Real')
plt.ylabel('Predicted')
plt.show()

In [ ]:
# 3.2.2 Perform validation model fit - A (LR in Statsmodels)
#trans_f = Pipeline(steps=('preprocessor',preprocessor_d))
#cat_vars_enc = trans_f.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names()
#cat_vars_enc
#con esto quiero saber el nombre de las variables q le puso para el statmodels
cat_vars_enc = encode_data_d.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names()
cat_vars_enc

In [ ]:
matrix_train = encode_data_d.fit_transform(X_train)
matrix_train #al pasar a una matriz numpy se pierde el indice

In [ ]:
selected_columns = ['age','bmi','children', 'male','smoker','northwest', 'southeast','southwest']
df_lr = pd.DataFrame(matrix_train,columns=selected_columns)
df_st = pd.concat([df_lr,y_train.reset_index(drop=True)],axis=1)
df_st

#vamos tratar de predecir los cargos


In [ ]:
formula = 'charges ~ age+bmi+children+male+smoker+northwest+southeast+southwest' #charge es un CL de age...southwest
result = smf.ols(formula=formula,data=df_st).fit()
print(result.summary())

#Jarque Bera compara contra una chi cuadrado con dos grados de libertad
#Un grado de la asimetria y el otro de la kurtosis

In [ ]:
res = result.resid #vemos los residuales
fig = qqplot(res,line='45',fit=True)
plt.show()

#no es tan normal. hay desviciones, lo q muetra la grafica.

In [ ]:
# 3.3 Perform model fit - B
#Vamos por la regresion de tipo arbol
tree_reg = Pipeline(steps=[('preprocessor',preprocessor_nd),('regressor',DecisionTreeRegressor())])
tree_reg.fig(X_train,y_train)
print(f"R2 Score:{tree_reg.score(X_train, y_train)}")

#da 1, sobre ajusta, estamos usando de un termino me dio 0.76 a algo q nos va a ir re mal, el arbl de regresion hizo demasiado overfit

In [ ]:
# 3.3.1 Calculate metrics - B
y_pred = tree_reg.predict(X_test)
print(f'R2 score: {r2_score(y_test,y_pred)}')
print(f'Mean squared error MSE: {mean_squared_error(y_test,y_pred)}')
print(f'Root Mean squared error RMSE: {np.sqrt(mean_squared_error(y_test,y_pred))}')

#se memorizo todo!, aca con datos nuevos manda el desvio 0.65 de r cuadrado, esta re mal
#el arbol no da bien

In [ ]:
plt.scatter(x=y_test,y=y_pred)
plt.plot([0,40000],[0,40000],color='Red')
plt.xlabel('Real')
plt.ylabel('Predicted')
plt.show()

In [ ]:
# 3.4 Perform model fit - C
#vamos con el random forest
rf_reg = Pipeline(steps=[('preprocessor',preprocessor_nd),('regressor',RandomForestRegressor())])
rf_reg.fig(X_train,y_train)
print(f"R2 Score:{rf_reg.score(X_train, y_train)}")

#este tambien sobre ajusto! le fue un poco mejor, paso a .81 hay algo de overfit

In [ ]:
# 3.4.1 Calculate metrics on model C
y_pred = rf_reg.predict(X_test)
print(f'R2 score: {r2_score(y_test,y_pred)}')
print(f'Mean squared error MSE: {mean_squared_error(y_test,y_pred)}')
print(f'Root Mean squared error RMSE: {np.sqrt(mean_squared_error(y_test,y_pred))}')

In [ ]:
plt.scatter(x=y_test,y=y_pred)
plt.plot([0,40000],[0,40000],color='Red')
plt.xlabel('Real')
plt.ylabel('Predicted')
plt.show()

In [ ]:
plt.scatter(x=y_test,y=y_pred)
plt.plot([0,50000],[0,50000],color='Red')
plt.xlabel('Real')
plt.ylabel('Predicted')
plt.show()

In [ ]:
# Step 4. Write up your conclusions

#Usamos el EDA y probamos 3 tipos de regresion (algoritmos)






